# Data Wrangling for Iowa Liquor Sales Database

Objective is to prepare this dataframe for the next stages of data analysis.
1. Correct data types. 
2. Matching duplicate pairs of data such as: store number/ store name 
3. Deal with missing values. 
4. Removing unneeded columns. 

In [1]:
# load packages

import pandas as pd
import numpy as np
import re

In [2]:
# CSV file is from "https://data.iowa.gov/Sales-Distribution/Iowa-Liquor-Sales/m3tr-qhgy"

# create variable to store dataset
iowa_liquor_sales = pd.read_csv('/Users/joe/Desktop/IOWA LIQUOR/Iowa_Liquor_Sales.csv')

/Users/joe/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (6,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# take a look at the data

iowa_liquor_sales.head().T

,0,1,2,3,4
Invoice/Item Number,INV-14268800019,INV-14257700018,INV-14217900044,INV-14268000158,INV-14312700006
Date,09/05/2018,09/05/2018,09/04/2018,09/05/2018,09/07/2018
Store Number,2508,5633,2590,3869,3880
Store Name,Hy-Vee Food Store #1 / Cedar Rapids,Home Town Wine & Spirits,Hy-Vee Food Store #5 / Cedar Rapids,Bootleggin' Barzini's Fin,Target Store T-2454 / Council Bluffs
Address,"1843 Johnson Avenue, N.W.",1202 A Ave E,3235 Oakland Road NE,412 1st Ave,3804 Metro Dr
City,Cedar Rapids,Oskaloosa,Cedar Rapids,Coralville,Council Bluffs
Zip Code,52405,52577,52402,52241,51503
Store Location,POINT (-91.697941 41.97447),POINT (-92.631487 41.296286),POINT (-91.658105 42.010971),POINT (-91.565517 41.672672),POINT (-95.837595 41.220979)
County Number,57.0,62.0,57.0,52.0,78.0
County,LINN,MAHASKA,LINN,JOHNSON,POTTAWATTA


### Observations
1. spelling differences to standardize
2. there are also some columns I have no interest in: invoice, volume of sales in gallons, address

In [4]:
# and it's size

iowa_liquor_sales.shape

(22972250, 24)

In [5]:
# 22M entries with 24 features

In [6]:
# checking for missing
iowa_liquor_sales.isna().sum()

Invoice/Item Number            0
Date                           0
Store Number                   0
Store Name                     0
Address                    80069
City                       80068
Zip Code                   80113
Store Location           2263587
County Number             156873
County                    156871
Category                   16974
Category Name              25040
Vendor Number                  9
Vendor Name                    7
Item Number                    0
Item Description               0
Pack                           0
Bottle Volume (ml)             0
State Bottle Cost             10
State Bottle Retail           10
Bottles Sold                   0
Sale (Dollars)                10
Volume Sold (Liters)           0
Volume Sold (Gallons)          0
dtype: int64

### Missing values thoughts

1. Store location, GPS information is missing around 11% (at the moment I am uncertain if I will use this information.)

2. A potential use for this information would be to determine a good location for a new store. GPS data is coordinates on a plane. Basic statistics could be used to determine distance between stores

### Open questions

1. Do stores groupped together see greater sales or does isolated stores see greater sales. We also have zip codes for each store which are missing far fewer entries. Store density per zip code could also give an idea of this information, especially if acres/ sq. miles per zip is available

2. Before we simply drop nulls we need to know: Are the NaN values specific to a certain location? Would dropping them under represent an area

3. To do this let's explore the location information we do have, zip code. Once zip code has been cleaned we will be able to tell if the missing data is concentrated to a certain area. If the missing data is concentrated we will need to decide how to proceed. Which perhaps means ignoring store location entirely

## Let the Wrangling Begin!

### [ ' Zip Code ' ]

In [7]:
# ZIP CODE
iowa_liquor_sales['Zip Code'].nunique()

956

### Additional information
An internet search states that iowa has 1055 zip codes. At least the number isn't larger. 

In [8]:
# zip code data type
iowa_liquor_sales['Zip Code'].dtypes

dtype('O')

### Objective: 

Change zipcode from object type to a numeric dtype

In [9]:
####### this code provides an error, but it let me know I had a non-numerical entry  ########

# iowaLiquorSales['Zip Code'] = iowaLiquorSales['Zip Code'].astype('float64')

# this let me know I need to look out for '712-2'

In [10]:
# how many have that entries have this faulty zip code?

iowa_liquor_sales[iowa_liquor_sales['Zip Code'] == '712-2'].groupby(['City']).describe()

Store Number                                                       \
              count    mean  std     min     25%     50%     75%     max   
City                                                                       
DUNLAP       6989.0  4307.0  0.0  4307.0  4307.0  4307.0  4307.0  4307.0   
Dunlap        951.0  4307.0  0.0  4307.0  4307.0  4307.0  4307.0  4307.0   

       County Number        ... Volume Sold (Liters)        \
               count  mean  ...                  75%   max   
City                        ...                              
DUNLAP        6604.0  43.0  ...                 10.5  60.0   
Dunlap         471.0  43.0  ...                  9.0  42.0   

       Volume Sold (Gallons)                                              \
                       count      mean       std   min   25%   50%   75%   
City                                                                       
DUNLAP                6989.0  1.440219  1.409358  0.03  0.26  0.59  2.77   
Dunlap                 951.0  1.369085  1.258462  0.05  0.40  0.92  2.38   

               
          max  
City           
DUNLAP  15.85  
Dunlap  11.10  

[2 rows x 96 columns]

### Observations

1. All of the entries are from the same city Dunlap. 
2. We also now know the 'city' feature has spelling issues

### Additional information

Dunlap which has a actual zip code of 51529 and turns out their area code is 712, likely the origin of the error

### Objective: 

Change all 712-2 zip codes to 51529

In [11]:
# replace all of those entries to the correct zip code

iowa_liquor_sales['Zip Code'] = iowa_liquor_sales['Zip Code'].replace({'712-2': 51529})

In [12]:
# examine different zip codes

iowa_liquor_sales['Zip Code'].unique()

array([52405, 52577, 52402, 52241, 51503, 50021, 50588, 50310, 50314,
       50501, 52245, 51041, 52240, 50023, 52722, 50322, 50035, 50315,
       50317, 52213, 50266, 50702, 50009, 50220, 52411, 50703, 52732,
       50854, 50644, 50010, 52807, 50613, 51501, 50265, 50135, 51351,
       50595, 50142, 50554, 50450, 51034, 51250, 50428, 52052, 51334,
       51566, 52804, 50129, 50616, 52002, 50313, 50701, 52001, 52501,
       52159, 50401, 52332, 52040, 51104, 52404, 51360, 50583, 50125,
       50707, 50247, 51401, 52205, 50111, 50312, 51103, 50158, 52172,
       50311, 51201, 52246, 51054, 51106, 50677, 50849, 52302, 50441,
       50321, 52175, 50138, 52060, 50014, 52401, 52803, 50581, 50423,
       51108, 50325, 52317, 52136, 50036, 50469, 50201, 52043, 50320,
       50638, 50058, 51534, 51301, 51105, 50126, 51031, 50219, 52544,
       50248, 52358, 52214, 51551, 52403, 52154, 50475, 50107, 52314,
       52031, 52208, 50211, 51247, 50112, 50621, 52003, 52057, 52748,
       50309, 50665,

### Observations
1. float values
2. int values
3. strings
4. one nan

### Additional thoughts
there is only 1 nan in 22M. we could easily drop it. with:
iowa_liquor_sales = iowa_liquor_sales[iowa_liquor_sales['Zip Code'].notna()]

But it is also just 1 so correcting it will be fast too.

In [13]:
# turn the nan into a zero
iowa_liquor_sales['Zip Code'] = iowa_liquor_sales['Zip Code'].replace(np.nan, 0)

# check out where that store location is
iowa_liquor_sales[iowa_liquor_sales['Zip Code'] == 0].groupby(['City']).describe()

Store Number                                                       \
               count    mean  std     min     25%     50%     75%     max   
City                                                                        
Stanton         45.0  5276.0  0.0  5276.0  5276.0  5276.0  5276.0  5276.0   

        County Number       ... Volume Sold (Liters)        \
                count mean  ...                  75%   max   
City                        ...                              
Stanton           0.0  NaN  ...                  4.5  10.5   

        Volume Sold (Gallons)                                                   
                        count      mean       std  min   25%   50%   75%   max  
City                                                                            
Stanton                  45.0  1.071333  0.738634  0.3  0.59  0.59  1.19  2.77  

[1 rows x 96 columns]

In [14]:
# Our single missing value is from stanton
# what is the zip code of stanton?
iowa_liquor_sales[iowa_liquor_sales['City'] == 'Stanton'].groupby(['Zip Code']).describe()

Store Number                                                       \
                count    mean  std     min     25%     50%     75%     max   
Zip Code                                                                     
0                45.0  5276.0  0.0  5276.0  5276.0  5276.0  5276.0  5276.0   
51573           725.0  5276.0  0.0  5276.0  5276.0  5276.0  5276.0  5276.0   
51573            68.0  5276.0  0.0  5276.0  5276.0  5276.0  5276.0  5276.0   

         County Number        ... Volume Sold (Liters)        \
                 count  mean  ...                  75%   max   
Zip Code                      ...                              
0                  0.0   NaN  ...                  4.5  10.5   
51573            725.0  69.0  ...                 10.5  63.0   
51573             68.0  69.0  ...                  9.0  21.0   

         Volume Sold (Gallons)                                              \
                         count      mean       std   min   25%   50%   75%   
Zip Code                                                                     
0                         45.0  1.071333  0.738634  0.30  0.59  0.59  1.19   
51573                    725.0  2.806262  2.264226  0.01  2.37  2.37  2.77   
51573                     68.0  1.913971  1.446331  0.59  0.79  1.19  2.38   

                 
            max  
Zip Code         
0          2.77  
51573     16.64  
51573      5.55  

[3 rows x 96 columns]

### Observation

1. We already knew about data type duplicates.
2. Stanton's zip code is 51573

In [15]:
# Objective: change zipcode '0' to the correct value of 51573

# earlier we used the replace method because the zipcode had a '-' in it. if we use it again:
# iowa_liquor_sales['Zip Code'] = iowa_liquor_sales['Zip Code'].replace({0: 51573})
# all zipcodes containing a '0' would have their '0' replaced with five extra digits

# instead we will write over those specific entries
iowa_liquor_sales.loc[iowa_liquor_sales['Zip Code'] == 0, 'Zip Code'] = 51573

In [16]:
# now the zip codes should be ready for conversion
# converting all zip codes to numeric
iowa_liquor_sales['Zip Code'] = iowa_liquor_sales['Zip Code'].astype('int64')

In [17]:
# I know it was a while ago, but all that zip code cleaning was to see if the missing store location data
# is concentrated.

# we can groupby zipcode to see if the missing gps locations are in the same place

df = iowa_liquor_sales[['Store Location', 'Zip Code']]

In [18]:
# get rid of the null values in dummy dataframe
df['Store Location'] = df['Store Location'].fillna(0)

/var/folders/70/gq6zfpsx5rg7k0vsvcbg70_40000gn/T/ipykernel_87008/2221258938.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Store Location'] = df['Store Location'].fillna(0)


In [19]:
# focus on just the missing locations
dff = df[df['Store Location'] == 0]

In [20]:
# how many missing values are there per zip code
dff.value_counts()


Store Location  Zip Code
0               50010       131355
                52804       123691
                52302       121012
                50701       115002
                50021       104475
                             ...  
                52778            1
                52625            1
                52151            1
                50638            1
                50006            1
Length: 255, dtype: int64

In [21]:
# missing store locations might be grouped. There are 100,000 missing in multiple zip codes
# remember there are about 2M missing store locations

iowa_liquor_sales[iowa_liquor_sales['Zip Code'] == 52302].groupby(['City']).describe()

Store Number                                                            \
              count         mean          std     min     25%     50%     75%   
City                                                                            
MARION      76434.0  3315.830455   792.705327  2560.0  2560.0  2847.0  4180.0   
Marion     124775.0  3607.472482  1031.075003  2514.0  2560.0  3868.0  4180.0   

                Zip Code           ... Volume Sold (Liters)          \
           max     count     mean  ...                  75%     max   
City                               ...                                
MARION  4736.0   76434.0  52302.0  ...                 10.5  1050.0   
Marion  6254.0  124775.0  52302.0  ...                 10.5  2100.0   

       Volume Sold (Gallons)                                              \
                       count      mean       std   min   25%   50%   75%   
City                                                                       
MARION               76434.0  2.139319  5.502494  0.03  0.59  1.27  2.77   
Marion              124775.0  2.204316  6.600444  0.00  0.59  1.38  2.77   

                
           max  
City            
MARION  277.38  
Marion  554.76  

[2 rows x 104 columns]

### Observations

1. Just from the most common missing gps data. Ames Iowa. 131355 out of the 559568 that is 20% which is over a years worth. I do not want to drop these missing values.
2. From the 5th most common missing, ankeny 104475 / ~250000 that is ~40%
3. 52302 (city of marion) has 121012 missing values  and about 200000 total which is 60%
4. The missing gps data appears to be correlated. I will be dropping store location.

In [22]:
# drop unneeded columns

# invoice and pack columns are important for the distributers with warehouse inventory, not for store habits.
# store sell the individual items not cases most often.

iowa = iowa_liquor_sales.drop(columns=['Invoice/Item Number','Address','Store Location','Pack','Volume Sold (Gallons)'])

In [23]:
# current state
iowa.head().T

,0,1,2,3,4
Date,09/05/2018,09/05/2018,09/04/2018,09/05/2018,09/07/2018
Store Number,2508,5633,2590,3869,3880
Store Name,Hy-Vee Food Store #1 / Cedar Rapids,Home Town Wine & Spirits,Hy-Vee Food Store #5 / Cedar Rapids,Bootleggin' Barzini's Fin,Target Store T-2454 / Council Bluffs
City,Cedar Rapids,Oskaloosa,Cedar Rapids,Coralville,Council Bluffs
Zip Code,52405,52577,52402,52241,51503
County Number,57.0,62.0,57.0,52.0,78.0
County,LINN,MAHASKA,LINN,JOHNSON,POTTAWATTA
Category,1081600.0,1081400.0,1081600.0,1031100.0,1011200.0
Category Name,Whiskey Liqueur,American Schnapps,Whiskey Liqueur,American Vodkas,Straight Bourbon Whiskies
Vendor Number,421.0,434.0,260.0,461.0,85.0


In [24]:
# check column data types
iowa.dtypes

Date                     object
Store Number              int64
Store Name               object
City                     object
Zip Code                  int64
County Number           float64
County                   object
Category                float64
Category Name            object
Vendor Number           float64
Vendor Name              object
Item Number              object
Item Description         object
Bottle Volume (ml)        int64
State Bottle Cost       float64
State Bottle Retail     float64
Bottles Sold              int64
Sale (Dollars)          float64
Volume Sold (Liters)    float64
dtype: object

In [25]:
# missing values
iowa.isna().sum()

Date                         0
Store Number                 0
Store Name                   0
City                     80068
Zip Code                     0
County Number           156873
County                  156871
Category                 16974
Category Name            25040
Vendor Number                9
Vendor Name                  7
Item Number                  0
Item Description             0
Bottle Volume (ml)           0
State Bottle Cost           10
State Bottle Retail         10
Bottles Sold                 0
Sale (Dollars)              10
Volume Sold (Liters)         0
dtype: int64

### [ ' Date ' ]

In [26]:
#DATE COLUMN
# we know that date is currently an object, we would like to have this be in datetime
iowa['Date'].describe(datetime_is_numeric=True)

count       22972250
unique          2502
top       12/22/2020
freq           18002
Name: Date, dtype: object

### Observation
1. 18000 deliveries on 12/22/2020. There are only ~2600 different storesthat is a busy day at the warehouse

In [27]:
# 'Date' column is also an object type. 
# let's change that to date time with pandas.to_datetime()

iowa['Date'] = pd.to_datetime(iowa['Date'])

### [ ' Store Number ' ] and [ ' Store Name ' ]

In [28]:
# STORE NUMBER COLUMN

# how many different store numbers are there?
iowa['Store Number'].nunique()

2687

In [29]:
# STORE NAME COLUMN

iowa['Store Name'].describe()

count                         22972250
unique                            2839
top       Hy-Vee #3 / BDI / Des Moines
freq                            191226
Name: Store Name, dtype: object

### Observation
1. 2839 unique store names
2. 2687 unique numbers

so there are at least some spelling issues

In [30]:
# clean up some

iowa['Store Name'] = iowa['Store Name'].str.capitalize()

In [31]:
# function to clean up duplicates
# we have more names than numbers, so numbers will have certainly have duplicates

def duplicate_cleaner(dataset, col_more, col_less):
    
    ### PART 1: find the duplicate values
    
    # create a dummy df with just the store number and name that has all the combinations
    dup_df1 = dataset.drop_duplicates(subset = [col_less, col_more])[[col_less, col_more]]

    # sort these combinations by occurances of the column with less values 
    dup_df2 = dup_df1.groupby(col_less).count()
    
    # ascending false because we want the larger numbers aka duplicates
    dup_df2 = dup_df2.sort_values(col_more, ascending = False)
    
    # remove singles, or non-duplicates
    dup_df3 = dup_df2[dup_df2[col_more] > 1 ].reset_index()
    
    ### PART 2: create a list of values for mapping
    
    # list of values
    values = dataset.groupby([col_less, col_more]).count().reset_index()
        
    # create a duplicate value list from the duplicate df
    wrong_values = values[values[col_less].isin(dup_df3[col_less])].sort_values(col_less)[[col_less, col_more]]
    
    # list of 'correct' names, most common name entry for each store
    values_map = wrong_values.groupby(col_less).max()[[col_more]].reset_index()
    
    ### PART 3: merge the correct values onto the original dataset, merged on col_less
    # and delete duplicate columns
    
    # merge 'correct' names with the df, merged on store number
    dataset = pd.merge(left = dataset, right = values_map, left_on = col_less, right_on = col_less, how = 'left')
    
    # now our dataframe has multiple store name columns
    # col_more_y is the cleaned values for duplicates
    
    # fill in the missing values (the values where there was no duplication from the original name column)
    dataset[col_more] = dataset[col_more+'_y'].fillna(dataset[col_more+'_x'])
    
    # drop the extra col_more column
    dataset = dataset.drop(columns=[col_more+'_x', col_more+'_y'])
    
    print(dataset[[col_less, col_more]].nunique())
    return dataset

In [32]:
# data, column less #, column more #
iowa = duplicate_cleaner(iowa, 'Store Name', 'Store Number')

Store Number    2687
Store Name      2555
dtype: int64


In [33]:
### We now have less Names than Numbers, so let's do it the other direction.
iowa = duplicate_cleaner(iowa, 'Store Number', 'Store Name')

Store Name      2555
Store Number    2555
dtype: int64


### [ ' City ' ]

#### From earlier we know that city has 80000 nan's

In [34]:
# CITY
iowa["City"].describe()

count       22892182
unique           846
top       Des Moines
freq         1166664
Name: City, dtype: object

### Observations
1. most common city is the largest city
2. 846 cities

We also likely have spelling issues

In [35]:
# check for any incorrect entries
# first let's standardize capitalization

# convert all cities to upper case
iowa['City'] = iowa['City'].str.upper()

In [36]:
# look for duplicates
dummy = iowa['City'].drop_duplicates()
sorted(dummy.astype('str'))

['ACKLEY',
 'ADAIR',
 'ADEL',
 'AFTON',
 'AKRON',
 'ALBERT CITY',
 'ALBIA',
 'ALBION',
 'ALBURNETT',
 'ALDEN',
 'ALGONA',
 'ALLERTON',
 'ALLISON',
 'ALTA',
 'ALTA VISTA',
 'ALTON',
 'ALTOONA',
 'ALVORD',
 'AMANA',
 'AMES',
 'ANAMOSA',
 'ANITA',
 'ANKENY',
 'ANTHON',
 'APLINGTON',
 'ARLINGTON',
 'ARMSTRONG',
 "ARNOLD'S PARK",
 'ARNOLDS PARK',
 'ATKINS',
 'ATLANTIC',
 'AUDUBON',
 'AURELIA',
 'AVOCA',
 'BALDWIN',
 'BANCROFT',
 'BAXTER',
 'BEDFORD',
 'BELLE PLAINE',
 'BELLEVUE',
 'BELMOND',
 'BETTENDORF',
 'BEVINGTON',
 'BLAIRSTOWN',
 'BLOOMFIELD',
 'BLUE GRASS',
 'BONDURANT',
 'BOONE',
 'BOYDEN',
 'BRITT',
 'BROOKLYN',
 'BUFFALO',
 'BUFFALO CENTER',
 'BURLINGTON',
 'BUSSEY',
 'CALMAR',
 'CAMANCHE',
 'CAMBRIDGE',
 'CARLISLE',
 'CARROLL',
 'CARTER LAKE',
 'CASCADE',
 'CASEY',
 'CEDAR FALLS',
 'CEDAR RAPIDS',
 'CENTER POINT',
 'CENTERVILLE',
 'CENTRAL CITY',
 'CHARITON',
 'CHARLES CITY',
 'CHEROKEE',
 'CHESTER',
 'CLARENCE',
 'CLARINDA',
 'CLARION',
 'CLARKSVILLE',
 'CLEAR LAKE',
 'CLEARLAKE

### Observations:
1. There are some duplicates that are almost spelled the same
2. there is a nan

In [37]:
# create a map to swap-out / correct the spelling
mapping = {"ARNOLD'S PARK" : 'ARNOLDS PARK',
           'CLEAR LAKE' : 'CLEARLAKE',
           'COLORADO SPRINGS' : np.nan,
           'FT. ATKINSON' : 'FORT ATKINSON',
           'GRAND MOUND' : 'GRAND MOUNDS',
           'GUTTENBERG' : 'GUTTENBURG',
           'KELLOG' : 'KELLOGG',
           'LECLAIRE' : 'LE CLAIRE',
           'LEMARS' : 'LE MARS',
           'MT PLEASANT' : 'MOUNT PLEASANT',
           'MT VERNON' : 'MOUNT VERNON',
           'OTTUWMA' : 'OTTUMWA',
           'OTUMWA' : 'OTTUMWA'}

In [38]:
#### map the correct spelllings onto their replacements
iowa['City'] = iowa['City'].replace(mapping.keys(), list(map(str, mapping.values())), regex=True)

In [39]:
# we are going to fill missing values with the preceeding value when sorted by that column (alphabetical)
# if we sort by zip then city all the na cities should be at the bottom and ffill will fill them mostly correctly

iowa = iowa.sort_values(by = ['Zip Code','City'])
iowa['City'] = iowa['City'].fillna(method='ffill')

### [ ' County Number ' ]

In [40]:
# COUNTY NUMBER

# this is an error
#iowa['County Number'] = iowa['County Number'].astype('int64')

# IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

iowa['County Number'] = iowa['County Number'].fillna(0)

In [41]:
# try this again
iowa['County Number'] = iowa['County Number'].astype('int64')

In [42]:
# check out some zip code combinations for the county 0
iowa.groupby('County Number').agg({'Zip Code': ['count','max','min',pd.Series.mode,pd.Series.nunique]})

Zip Code                             
                 count    max    min   mode nunique
County Number                                      
0               156873  80904  50009  51573      98
1                46923  50849  50002  50250       4
2                18669  50841  50841  50841       1
3                94753  52172  52146  52172       4
4                82291  52571  52544  52544       2
...                ...    ...    ...    ...     ...
95               79682  50450  50424  50436       3
96              117014  52161  52101  52101       4
97              713633  51109  51004  51106      12
98               32251  50459  50456  50459       2
99               66076  50542  50071  50533       5

[100 rows x 5 columns]

In [43]:
# 98 different zip codes are represented in the missing county values
# one, 80904, appears to not be in iowa
iowa = iowa[iowa['County Number'] != 0]

In [44]:
iowa.isna().sum()

Date                        0
City                        0
Zip Code                    0
County Number               0
County                      0
Category                16748
Category Name           24644
Vendor Number               9
Vendor Name                 7
Item Number                 0
Item Description            0
Bottle Volume (ml)          0
State Bottle Cost          10
State Bottle Retail        10
Bottles Sold                0
Sale (Dollars)             10
Volume Sold (Liters)        0
Store Name                  0
Store Number                0
dtype: int64

In [45]:
# a quick internet search of: number of counties in iowa, 99 is correct

iowa['County Number'].nunique()

99

### [ ' County ' ]

In [46]:
# COUNTY

# we know it is supposed to be 99
iowa['County'].nunique()

200

In [47]:
# 200 > 99. Examine some values

# make all county names capitalized
iowa['County'] = iowa['County'].str.upper()

# did we catch 'em all?
iowa['County'].nunique()

103

In [48]:
# look for duplicates
dummy = iowa['County'].drop_duplicates()
sorted(dummy.astype('str'))

['ADAIR',
 'ADAMS',
 'ALLAMAKEE',
 'APPANOOSE',
 'AUDUBON',
 'BENTON',
 'BLACK HAWK',
 'BOONE',
 'BREMER',
 'BUCHANAN',
 'BUENA VIST',
 'BUENA VISTA',
 'BUTLER',
 'CALHOUN',
 'CARROLL',
 'CASS',
 'CEDAR',
 'CERRO GORD',
 'CERRO GORDO',
 'CHEROKEE',
 'CHICKASAW',
 'CLARKE',
 'CLAY',
 'CLAYTON',
 'CLINTON',
 'CRAWFORD',
 'DALLAS',
 'DAVIS',
 'DECATUR',
 'DELAWARE',
 'DES MOINES',
 'DICKINSON',
 'DUBUQUE',
 'EMMET',
 'FAYETTE',
 'FLOYD',
 'FRANKLIN',
 'FREMONT',
 'GREENE',
 'GRUNDY',
 'GUTHRIE',
 'HAMILTON',
 'HANCOCK',
 'HARDIN',
 'HARRISON',
 'HENRY',
 'HOWARD',
 'HUMBOLDT',
 'IDA',
 'IOWA',
 'JACKSON',
 'JASPER',
 'JEFFERSON',
 'JOHNSON',
 'JONES',
 'KEOKUK',
 'KOSSUTH',
 'LEE',
 'LINN',
 'LOUISA',
 'LUCAS',
 'LYON',
 'MADISON',
 'MAHASKA',
 'MARION',
 'MARSHALL',
 'MILLS',
 'MITCHELL',
 'MONONA',
 'MONROE',
 'MONTGOMERY',
 'MUSCATINE',
 "O'BRIEN",
 'OBRIEN',
 'OSCEOLA',
 'PAGE',
 'PALO ALTO',
 'PLYMOUTH',
 'POCAHONTAS',
 'POLK',
 'POTTAWATTA',
 'POTTAWATTAMIE',
 'POWESHIEK',
 'RINGGOL

In [49]:
# So we have 4 extra.

# only four so let's just manually fix them
iowa.loc[iowa['County'] == 'BUENA VIST', 'County'] = 'BUENA VISTA'
iowa.loc[iowa['County'] == 'CERRO GORD', 'County'] = 'CERRO GORDO'
iowa.loc[iowa['County'] == "OBRIEN", 'County'] = "O'BRIEN"
iowa.loc[iowa['County'] == 'POTTAWATTA', 'County'] = 'POTTAWATTAMIE'

In [50]:
# is it 99?
iowa['County'].nunique()

99

### [ ' Category ' ]  &  [ ' Category Name ' ]

In [51]:
# CATEGORY

# try standardizing letter case
iowa['Category Name'] = iowa['Category Name'].str.upper()
iowa['Category Name'].nunique()

112

In [52]:
iowa.groupby(['Category', 'Category Name'])['Category'].count()

Category   Category Name                 
1011000.0  AMERICAN WHISKIES                       1
1011100.0  BLENDED WHISKIES                   990691
1011200.0  STRAIGHT BOURBON WHISKIES         1391105
1011250.0  SINGLE BARREL BOURBON WHISKIES       9205
1011300.0  SINGLE BARREL BOURBON WHISKIES      18946
                                              ...   
1701100.0  TEMPORARY & SPECIALTY PACKAGES     160266
1701200.0  HOLIDAY VAP                            48
1900000.0  DELISTED / SPECIAL ORDER ITEMS          1
1901100.0  DELISTED ITEMS                         63
1901200.0  SPECIAL ORDER ITEMS                 26815
Name: Category, Length: 129, dtype: int64

In [53]:
# AMERICAN WHISKIES has one entry? 
iowa[iowa['Category Name'] =='AMERICAN WHISKIES']

,Date,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Store Name,Store Number
19237037,2021-09-23,DES MOINES,50320,77,POLK,1011000.0,AMERICAN WHISKIES,566.0,"Foundry Distilling Company, LLC",926577,Glen Oaks Whiskey Buy the Barrel,750,19.0,28.5,66,1881.0,49.5,Hy-vee #3 / bdi / des moines,2633.0


They appear to have bought a barrel, so only one entry makes sense



In [54]:
# look for duplicates
dummy = iowa['Category'].drop_duplicates()
sorted(dummy.astype('str'))

['1011000.0',
 '1011100.0',
 '1011200.0',
 '1011250.0',
 '1011300.0',
 '1011400.0',
 '1011500.0',
 '1011600.0',
 '1011700.0',
 '1011800.0',
 '1012000.0',
 '1012100.0',
 '101220.0',
 '1012200.0',
 '1012210.0',
 '1012300.0',
 '1012400.0',
 '1022100.0',
 '1022200.0',
 '1022300.0',
 '1031000.0',
 '1031080.0',
 '1031090.0',
 '1031100.0',
 '1031110.0',
 '1031200.0',
 '1032000.0',
 '1032080.0',
 '1032100.0',
 '1032200.0',
 '1032230.0',
 '1041100.0',
 '1041150.0',
 '1041200.0',
 '1041300.0',
 '1042000.0',
 '1042100.0',
 '1051010.0',
 '1051100.0',
 '1051110.0',
 '1051120.0',
 '1051140.0',
 '1051150.0',
 '1052010.0',
 '1052100.0',
 '1062050.0',
 '1062100.0',
 '1062200.0',
 '1062250.0',
 '1062300.0',
 '1062310.0',
 '1062400.0',
 '1062500.0',
 '1070000.0',
 '1070100.0',
 '1071100.0',
 '1081000.0',
 '1081010.0',
 '1081015.0',
 '1081020.0',
 '1081030.0',
 '1081100.0',
 '1081200.0',
 '1081210.0',
 '1081220.0',
 '1081230.0',
 '1081240.0',
 '1081250.0',
 '1081300.0',
 '1081305.0',
 '1081312.0',
 '10813

In [55]:
# This category as one fewer character
iowa[iowa['Category'] == 101220.0]

,Date,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Store Name,Store Number
9127780,2013-12-12,DES MOINES,50312,77,POLK,101220.0,NaN,885.0,"Yahara Bay Distillers, Inc",901369,Craigellachie,750,51.52,77.29,6,463.74,4.50,Vom fass / des moines,4669.0
9303210,2013-10-24,DES MOINES,50312,77,POLK,101220.0,NaN,885.0,"Yahara Bay Distillers, Inc",901369,Craigellachie,750,51.52,77.29,3,231.87,2.25,Vom fass / des moines,4669.0
10650130,2013-09-04,DES MOINES,50312,77,POLK,101220.0,NaN,885.0,"Yahara Bay Distillers, Inc",901369,Craigellachie,750,51.52,77.29,3,231.87,2.25,Vom fass / des moines,4669.0
11699978,2013-05-20,DES MOINES,50312,77,POLK,101220.0,NaN,885.0,"Yahara Bay Distillers, Inc",901369,Craigellachie,750,51.52,77.29,3,231.87,2.25,Vom fass / des moines,4669.0
13445600,2014-04-17,DES MOINES,50312,77,POLK,101220.0,NaN,885.0,"Yahara Bay Distillers, Inc",901369,Craigellachie,750,52.37,78.56,3,235.68,2.25,Vom fass / des moines,4669.0
13489741,2014-08-20,DES MOINES,50312,77,POLK,101220.0,NaN,885.0,"Yahara Bay Distillers, Inc",901369,Craigellachie,750,52.37,78.56,3,235.68,2.25,Vom fass / des moines,4669.0


### Observations:
1. Category name is missing
2. The item description tells me it is a scotch

In [56]:
# 101220.0 is scotch so let's fix that
# find the category number for scotch
iowa.loc[iowa['Category Name'] == 'SCOTCH WHISKIES', 'Category']

5448209     1012200.0
6406623     1012200.0
9547126     1012200.0
16681128    1012200.0
19108560    1012200.0
              ...    
18457438    1012200.0
19829904    1012200.0
18421548    1012200.0
19678746    1012200.0
20842553    1012200.0
Name: Category, Length: 405446, dtype: float64

In [57]:
# fix the incorrect scotch label
iowa.loc[iowa['Category'] == 101220.0, 'Category'] = 1012200.0

In [58]:
# clean up missing category name
iowa['Category Name'] = iowa['Category Name'].fillna(0)

In [59]:
# clean up missing category
iowa['Category'] = iowa['Category'].fillna(0)

In [60]:
# we filled nan values with 0 so we need to make those 0 --> '0'
iowa['Category Name'] = iowa['Category Name'].astype('str')

In [61]:
# let's clean up some Category names

# apply re to all column entries
def regularexpression(x):
    
    # apply the regular expression
    # removing all non alpha characters or spaces
    return re.sub(r'[^a-zA-Z ]', ' ', x)  

In [62]:
# Let's apply the RE to the category name column
iowa['Category Name'] = iowa['Category Name'].apply(lambda x: regularexpression(x))

In [63]:
# condense some categories by removing extra information

words_to_remove = ['AMERICAN', 'IMPORTED', 'STRAIGHT', 'DRY', 'PROOF', 'GRAPE'] 

In [64]:
for word in words_to_remove: 
    iowa['Category Name'] = iowa['Category Name'].str.replace(word ,'')

In [65]:
## the nan column is now and empty string

### first look throughthere are a bunch of over laps
# create a map to swap-out / correct the spelling
CATE_MAP = {'' : 'EMPTY',
           'GINS' : 'GIN',
           'VODKAS' : 'VODKA',
           'LIQUEURS' : 'LIQUEUR',
           'SPIRITS' : 'SPIRIT',
           'WHISKIES' : 'WHISKY',
           'WHISKEY' : 'WHISKY',
           'VODKA  MISC' : 'VODKA FLAVORED',
           'TENNESSEE WHISKIES' : 'WHISKY',
           'AGAVE TEQUILA' : 'TEQUILA',
           'VODKA  CHERRY' : 'VODKA FLAVORED',
           'WHITE CREME DE CACAO' : 'LIQUEUR',
           'WHITE CREME DE MENTHE' : 'LIQUEUR'}

## there are some double spaces that will be taken care of last. Because we still might generate some more

In [66]:
#### map the correct spelllings onto their replacements
iowa['Category Name'] = iowa['Category Name'].replace(CATE_MAP.keys(), list(map(str, CATE_MAP.values())), regex=True)

In [67]:
# create a function to replace values

def type(x):
    #search through values to apply a new label
    if "IOWA" in x:
        return "IOWA LOCAL"
    elif "TEMPORARY" in x:
        return "SPECIAL PACKAGING"
    elif "HOLIDAY" in x:
        return "SPECIAL PACKAGING"
    elif 'SPECIALTY' in x:
        return "SPECIAL PACKAGING" 
    elif 'DISTILLED SPIRIT SPECIALTY' in x:
        return 'FLAVORED WHISKY'
    elif 'WHISKY LIQUEUR' in x:
        return 'FLAVORED WHISKY'
    elif "DELISTED" in x:
        return "DELISTED"
    elif "BRANDIES" in x:
        return "BRANDY"
    elif "SCHNAPPS" in x:
        return "SCHNAPPS"
    elif 'VODKA FLAVORED' in x:
        return 'FLAVORED VODKA'
    elif "FLAVORED" in x:
        return x
    elif 'ROCK  RYE' in x:
        return 'COCKTAILS RTD'
    elif "RYE" in x:
        return "RYE WHISKY"
    elif "CANADIAN WHISKY" in x:
        return x
    elif "CREME" in x:
        return "LIQUEUR"
    elif "SCOTCH" in x:
        return "SCOTCH"
    elif "BOURBON" in x:
        return "BOURBON WHISKY"
    elif "WHISK" in x:
        return "WHISKY"
    elif "LIQUEUR" in x:
        return "LIQUEUR"
    elif "AMARETTO" in x:
        return "LIQUEUR"
    elif 'TRIPLE SEC' in x:
        return "LIQUEUR"
    elif 'SLOE' in x:
        return "LIQUEUR"
    elif "RUM" in x:
        return "RUM"
    elif "MEZCAL" in x:
        return "MEZCAL" 
    elif "VODKA" in x:
        return "VODKA"
    elif "GIN" in x:
        return "GIN"
    elif 'NEUTRAL GRAIN SPIRIT FLAVORED' in x:
        return "FLAVORED WHISKY"
    elif "NEUTRAL" in x:
        return "NEUTRAL GRAIN"
    elif "ALCOHOL" in x:
        return "NEUTRAL GRAIN"
    elif "SPECIAL" in x:
        return "SPECIALTY"
    elif "COCKTAIL" in x:
        return 'COCKTAILS RTD'
    elif 'MIXTO TEQUILA' in x:
        return 'TEQUILA'
    elif 'ANISETTE' in x:
        return 'LIQUEUR'
    else:
        return x

In [68]:
# apply that function to create a new column
iowa['Category Name'] = iowa['Category Name'].apply(lambda x: type(x))

In [69]:
# remove the empty spaces
iowa['Category Name'] = iowa['Category Name'].str.strip()
iowa['Category Name'] = iowa['Category Name'].str.replace('  ',' ')

In [70]:
iowa = iowa[iowa['Category Name'] != 'EMPTY']
iowa = iowa[iowa['Category Name'] != 'HIGH BEER AMERICAN']
iowa = iowa[iowa['Category Name'] != 'DELISTED']

In [71]:
# look for easy over laps
dummy = iowa['Category Name'].drop_duplicates()
sorted(dummy.astype('str'))

['',
 'BOURBON WHISKY',
 'BRANDY',
 'CANADIAN WHISKY',
 'COCKTAILS RTD',
 'FLAVORED GIN',
 'FLAVORED RUM',
 'FLAVORED VODKA',
 'FLAVORED WHISKY',
 'GIN',
 'HIGH BEER',
 'IOWA LOCAL',
 'LIQUEUR',
 'MEZCAL',
 'NEUTRAL GRAIN',
 'NEUTRAL GRAIN SPIRIT FLAVORED',
 'RUM',
 'RYE WHISKY',
 'SCHNAPPS',
 'SCOTCH',
 'SPECIAL PACKAGING',
 'SPECIALTY',
 'TEQUILA',
 'VODKA',
 'WHISKY']

In [72]:
iowa = iowa[iowa['Category Name'] != 'nan']

In [73]:
# where are we at now
iowa[['Category', 'Category Name']].nunique()

Category         110
Category Name     25
dtype: int64

In [76]:
# duplicate cleaner function again
iowa = duplicate_cleaner(iowa, 'Category', 'Category Name')

Category Name    25
Category         25
dtype: int64


### [ ' Vendor Number ' ]  &  [ ' Vendor Name ' ] 

In [77]:
# VENDOR NUMBER & NAME

# I am less interested in vendor name/number. 
iowa[['Vendor Number', 'Vendor Name']].nunique()

Vendor Number    395
Vendor Name      546
dtype: int64

In [78]:
#standardize 
iowa['Vendor Name'] = iowa['Vendor Name'].str.upper()

In [79]:
# use the duplicate pair finding function
iowa = duplicate_cleaner(iowa, 'Vendor Name', 'Vendor Number')

Vendor Number    395
Vendor Name      396
dtype: int64


### Obeservation:
One name was not corrected. Is it nan?

In [80]:
iowa['Vendor Name'].isna().sum()

7

In [81]:
# Remove the 7 nulls
iowa = iowa[iowa['Vendor Name'] != np.nan]

In [91]:
iowa.groupby('Vendor Name')['Vendor Number'].nunique().sort_values().reset_index()

,Vendor Name,Vendor Number
0,RESERVOIR DISTILLERY,0
1,10TH MOUNTAIN WHISKEY & SPIRIT COMPANY,1
2,PURPLE VALLEY IMPORTS / GLASS REVOLUTION IMPORTS,1
3,"PUENTE INTERNATIONAL, INC.",1
4,PROXIMO,1
...,...,...
391,"GLOBAL BEVERAGE TEAM, LLC",1
392,GLASS REVOLUTION IMPORTS,1
393,GEORGETOWN TRADING CO. LLC / JAMES PEPPER DIST...,1
394,GARGOYLE ENTERPRISES / LAWS WHISKEY HOUSE,1


### That was not the result I was expecting.
What does Reservoir distillery sell?

In [92]:
iowa[iowa['Vendor Name'] == 'RESERVOIR DISTILLERY']

,Date,City,Zip Code,County Number,County,Vendor Number,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Store Name,Store Number,Category Name,Category,Vendor Name
5127038,2016-10-17,DES MOINES,50314,77,POLK,NaN,916456,Reservoir Bourbon Whiskey,750,45.0,67.5,30,2025.0,22.5,"Central city liquor, inc.",2190.0,WHISKY,1012400.0,RESERVOIR DISTILLERY
5127213,2016-10-17,DES MOINES,50314,77,POLK,NaN,927565,Reservoir Wheat Whiskey,750,45.0,67.5,18,1215.0,13.5,"Central city liquor, inc.",2190.0,SPECIAL PACKAGING,1701200.0,RESERVOIR DISTILLERY


#### Let's get them a number

In [93]:
iowa['Vendor Number'].max()

987.0

In [94]:
# max number is 987 so 988 must be available
iowa.loc[iowa['Vendor Name'] == 'RESERVOIR DISTILLERY', 'Vendor Number'] = 988.0

### Bottle, sales, volume of sales are in part 2

In [97]:
# drop unneeded columns
iowa = iowa.drop(columns=['Vendor Number', 'County Number', 'Store Number'])

In [98]:
# save progress as csv
iowa.to_csv('iowa_clean.csv', index=False)

In [99]:
# create internet sized sample
iowa_sample = iowa.sample(1000)

In [100]:
# save the sample as a csv
iowa_sample.to_csv('iowa_sample.csv', index=False)